<a href="https://colab.research.google.com/github/jayyanar/agentic-ai-training/blob/lab-day-2/labs-day-2/Lab2_1_RAG_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2.1: RAG Fundamentals with Groq & LangChain

In this lab, we will build a basic Retrieval-Augmented Generation (RAG) system for a **Wells Fargo Internal Knowledge Assistant**. The system will answer employee questions about internal banking policies using:
- **LLM**: Groq (Llama 3)
- **Embeddings**: HuggingFace (via `sentence-transformers`)
- **Vector Store**: ChromaDB
- **Framework**: LangChain

## Objectives
1. Setup environment and API keys.
2. Load internal banking policy documents.
3. Create vector embeddings and store them.
4. Query the banking knowledge base.

In [12]:
# 1. Install Dependencies
print("Installing dependencies...")
%pip install -qU langchain langchain-groq langchain-community langchain-huggingface chromadb sentence-transformers langsmith
print("Dependencies installed.")

Installing dependencies...
Dependencies installed.


In [13]:
# 2. Setup API Keys
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
# Optional for LangSmith Tracking
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "RAG Basics"

## 2. Load and Process Documents
We will load simulated internal banking policies.

In [14]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Simulated Internal Banking Policies (Wells Fargo Context)
banking_policy_text = """
Wells Fargo Internal Policy: Secure Access & Data Protection
Effective Date: January 2025

1. Access Control Guidelines
All employees must use Multi-Factor Authentication (MFA) to access internal financial systems. Passwords must be updated every 90 days.
Sharing of credentials is strictly prohibited and will result in immediate disciplinary action.

2. Customer Transaction Handling
Any transaction exceeding $10,000 must be reported under the Currency Transaction Report (CTR) guidelines.
Customer verification (KYC) is mandatory for all new account openings. Documents accepted include Passport, Driver's License, or State ID.

3. Fraud Detection Protocols
If suspicious activity is detected on a client account, the account must be temporarily frozen, and a 'Suspicious Activity Report' (SAR) must be filed within 24 hours.
Common indicators of fraud include: frequent small transactions followed by a large one, accessing accounts from high-risk geolocations, and multiple failed login attempts.

4. Remote Work Policy
Employees working remotely must use the company-provided VPN. Accessing client data from public Wi-Fi networks (e.g., cafes, airports) is strictly forbidden.
"""

# Create a Document object directly
docs = [Document(page_content=banking_policy_text, metadata={"source": "internal_policy_doc"})]

# Check content
print(f"Loaded {len(docs)} document(s).")
print(docs[0].page_content[:500])

Loaded 1 document(s).

Wells Fargo Internal Policy: Secure Access & Data Protection
Effective Date: January 2025

1. Access Control Guidelines
All employees must use Multi-Factor Authentication (MFA) to access internal financial systems. Passwords must be updated every 90 days.
Sharing of credentials is strictly prohibited and will result in immediate disciplinary action.

2. Customer Transaction Handling
Any transaction exceeding $10,000 must be reported under the Currency Transaction Report (CTR) guidelines.
Custom


In [15]:
# Split text into chunks (Using smaller chunks for this demo)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)
splits = text_splitter.split_documents(docs)
print(f"Split into {len(splits)} chunks.")

Split into 6 chunks.


## 3. Embeddings & Vector Store
We will use `HuggingFaceEmbeddings` (runs locally/free) and `Chroma` document store.

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create Vector Store
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings
)

# Create a retriever interface
retriever = vectorstore.as_retriever()
print("Vector store created and retriever ready.")

Vector store created and retriever ready.


In [17]:
# Let's test the retriever to see what it finds before feeding it to the LLM
print("--- Testing Retriever ---")
question = "What is the reporting requirement for large transactions?"
print(f"Query: {question}")

retrieved_docs = retriever.invoke(question)
for i, doc in enumerate(retrieved_docs):
    print(f"\n[Document {i+1} Source: {doc.metadata['source']}]\nContent: {doc.page_content}...")

print("-------------------------")

--- Testing Retriever ---
Query: What is the reporting requirement for large transactions?

[Document 1 Source: internal_policy_doc]
Content: 2. Customer Transaction Handling
Any transaction exceeding $10,000 must be reported under the Currency Transaction Report (CTR) guidelines.
Customer verification (KYC) is mandatory for all new account openings. Documents accepted include Passport, Driver's License, or State ID....

[Document 2 Source: internal_policy_doc]
Content: 2. Customer Transaction Handling
Any transaction exceeding $10,000 must be reported under the Currency Transaction Report (CTR) guidelines.
Customer verification (KYC) is mandatory for all new account openings. Documents accepted include Passport, Driver's License, or State ID....

[Document 3 Source: internal_policy_doc]
Content: Common indicators of fraud include: frequent small transactions followed by a large one, accessing accounts from high-risk geolocations, and multiple failed login attempts....

[Document 4 Sou

## 4. Query with Groq
Now we create a RAG chain to answer questions based on the document.

In [18]:
from langchain_groq import ChatGroq

# Initialize LLM
print("Initializing LLM...")
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)
print("LLM Initialized.")

Initializing LLM...
LLM Initialized.


In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# System Prompt
system_prompt = (
    "You are a Wells Fargo internal banking assistant. "
    "Use the following pieces of retrieved banking regulations to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer professional and concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

def format_docs(docs):
    """Combine the content of all retrieved documents into one string."""
    return "\n\n".join(doc.page_content for doc in docs)

# Create Chain
rag_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("RAG Chain successfully created.")

RAG Chain successfully created.


In [20]:
# Run Query
question = "What is the reporting requirement for large transactions?"
print(f"Question: {question}")
print("Generating Answer...")

response = rag_chain.invoke(question)
print("\nAnswer:", response)

Question: What is the reporting requirement for large transactions?
Generating Answer...

Answer: Transactions exceeding $10,000 must be reported under the Currency Transaction Report (CTR) guidelines. This applies to any single transaction or aggregate transactions totaling over $10,000 in a single business day. The report must include details such as the customer’s identity, transaction amount, and purpose.


## Exercise
Try changing the URL to a different article or PDF and see how the answers change!